#### Import modules

In [20]:
%matplotlib inline

import time
import random
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.metrics import roc_curve, auc
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from sklearn.model_selection import KFold, cross_val_score
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.ensemble import RandomForestClassifier 
from urllib.request import urlopen 

plt.style.use('ggplot')
pd.set_option('display.max_columns', 500) 

#### Read Data

In [39]:
financial_distress_data = pd.read_csv(r'C:\Users\mohitgan\Downloads\Additional Materials to be put in Pen Drive\Additional Courses Material\Extra Machine Learning Practice\Financial_Distress.csv')
financial_distress_data.head()

,Company,Time,Financial Distress,x1,x2,x3,x4,x5,x6,x7,x8,x9,x10,x11,x12,x13,x14,x15,x16,x17,x18,x19,x20,x21,x22,x23,x24,x25,x26,x27,x28,x29,x30,x31,x32,x33,x34,x35,x36,x37,x38,x39,x40,x41,x42,x43,x44,x45,x46,x47,x48,x49,x50,x51,x52,x53,x54,x55,x56,x57,x58,x59,x60,x61,x62,x63,x64,x65,x66,x67,x68,x69,x70,x71,x72,x73,x74,x75,x76,x77,x78,x79,x80,x81,x82,x83
0,1,1,0.010636,1.2810,0.022934,0.87454,1.21640,0.060940,0.188270,0.52510,0.018854,0.182790,0.006449,0.85822,2.00580,0.125460,6.9706,4.6512,0.050100,2.1984,0.018265,0.024978,0.027264,1.41730,9.5554,0.148720,0.66995,214.760,12.641,6.4607,0.043835,0.204590,0.35179,8.3161,0.28922,0.76606,2.5825,77.400,0.026722,1.63070,0.015016,0.005478,0.12730,9.6951,-0.73622,0.98559,0.180160,1.50060,0.026224,7.0513,1174.90,5.3399,0.85128,12.837,0.061737,0.180900,209.87,-0.582550,0.47101,0.109900,0.000000,0.000000,0.22009,7.1241,15.3810,3.2702,17.8720,34.6920,30.087,12.8,7991.4,364.9500,15.8,61.476,4.0,36.0,85.437,27.07,26.102,16.000,16.0,0.2,22,0.060390,30,49
1,1,2,-0.455970,1.2700,0.006454,0.82067,1.00490,-0.014080,0.181040,0.62288,0.006423,0.035991,0.001795,0.85152,-0.48644,0.179330,4.5764,3.7521,-0.014011,2.4575,0.027558,0.028804,0.041102,1.18010,7.2952,0.056026,0.67048,38.242,12.877,5.5506,0.265480,0.150190,0.41763,9.5276,0.41561,0.81699,2.6033,95.947,0.007580,0.83754,0.027425,0.045434,0.13774,5.6035,-0.64385,1.30190,0.046857,1.00950,0.007864,4.6022,1062.50,3.7389,0.94397,12.881,-0.000565,0.056298,250.14,-0.474770,0.38599,0.369330,0.000000,0.000000,0.00000,7.4166,7.1050,14.3210,18.7700,124.7600,26.124,11.8,8322.8,0.1896,15.6,24.579,0.0,36.0,107.090,31.31,30.194,17.000,16.0,0.4,22,0.010636,31,50
2,1,3,-0.325390,1.0529,-0.059379,0.92242,0.72926,0.020476,0.044865,0.43292,-0.081423,-0.765400,-0.054324,0.89314,0.41220,0.077578,11.8900,2.4884,0.028077,1.3957,0.012595,0.068116,0.014847,0.81652,7.1204,0.065220,0.84827,-498.390,13.225,16.2540,0.416570,0.074149,0.36723,9.3513,0.50356,0.91962,1.4931,144.670,-0.066483,0.95579,0.017270,0.028059,0.10242,9.4003,-14.03200,0.75746,-0.579760,0.57832,-0.064373,11.9880,651.15,10.9340,0.93478,12.909,0.041625,0.047562,280.55,-1.000000,0.48844,0.053299,0.003785,0.005191,0.00000,3.6373,7.0213,1.1538,9.8951,6.4467,30.245,10.3,8747.0,11.9460,15.2,20.700,0.0,35.0,120.870,36.07,35.273,17.000,15.0,-0.2,22,-0.455970,32,51
3,1,4,-0.566570,1.1131,-0.015229,0.85888,0.80974,0.076037,0.091033,0.67546,-0.018807,-0.107910,-0.065316,0.89581,0.99490,0.141120,6.0862,1.6382,0.093904,2.0588,0.011601,0.094385,0.014415,0.90391,7.9828,0.125160,0.80478,-75.867,13.305,8.8950,0.083774,0.054098,0.54360,7.0909,0.67133,0.93701,2.3533,219.750,-0.017000,0.38335,0.014327,0.203370,0.10143,5.7379,0.72205,1.39120,-0.150130,0.64508,-0.017731,6.1114,703.04,5.7028,0.87484,13.094,0.108400,0.101350,413.74,0.565000,0.34408,0.073356,0.000037,0.000045,0.00000,5.1442,9.9099,2.0408,-1.4903,-21.9070,34.285,11.5,9042.5,-18.7480,10.4,47.429,4.0,33.0,54.806,39.80,38.377,17.167,16.0,5.6,22,-0.325390,33,52
4,2,1,1.357300,1.0623,0.107020,0.81460,0.83593,0.199960,0.047800,0.74200,0.128030,0.577250,0.094075,0.81549,3.01470,0.185400,4.3938,1.6169,0.239210,3.0311,0.006814,0.079346,0.008876,1.02510,4.7463,0.266020,0.76770,1423.100,11.575,17.4880,0.620770,0.046907,0.56963,9.4861,0.68143,0.94242,4.1296,222.650,0.131230,0.25301,0.008152,0.353010,0.17612,4.5088,-0.11339,1.05270,0.607660,0.25782,0.131380,4.4151,2465.40,4.1408,0.73398,11.396,0.250310,0.222370,315.34,-0.060101,0.20242,1.229100,-0.002491,-0.002980,0.22688,7.1241,15.3810,3.2702,17.8720,34.6920,30.087,12.8,7991.4,364.9500,15.8,61.476,4.0,36.0,85.437,27.07,26.102,16.000,16.0,0.2,29,1.251000,7,27


#### Description

In [40]:
# First column: Company represents sample companies.

# Second column: Time shows different time periods that data belongs to. Time series length varies between 1 to 14 for each company.

# Third column: The target variable is denoted by "Financial Distress" if it is greater than -0.50 the company should be considered as healthy (0). 
# Otherwise, it would be regarded as financially distressed (1).

# Fourth column to the last column: The features denoted by x1 to x83, are some financial and non-financial characteristics of the sampled companies. 
# These features belong to the previous time period, which should be used to predict whether the company will be financially distressed or not (classification). 
# Feature x80 is a categorical variable.

#### Clean up of data

In [41]:
# Dropping 'Time' and Setting 'Company' as our index
del financial_distress_data['Time']
financial_distress_data.set_index(['Company'], inplace = True)

In [42]:
# Converted to binary to help later on with models and plots
# financial_distress['Financial Distress'] = financial_distress['Financial Distress'].map({'M':1, 'B':0})
def financial_distress(c):
    if c['Financial Distress'] > -0.5:
        return 0
    elif c['Financial Distress'] <= -0.5:
        return 1
    else:
        return 'Undefined'

financial_distress_data['Financial Distress'] = financial_distress_data.apply(financial_distress, axis=1)

In [43]:
financial_distress_data.head()

,Financial Distress,x1,x2,x3,x4,x5,x6,x7,x8,x9,x10,x11,x12,x13,x14,x15,x16,x17,x18,x19,x20,x21,x22,x23,x24,x25,x26,x27,x28,x29,x30,x31,x32,x33,x34,x35,x36,x37,x38,x39,x40,x41,x42,x43,x44,x45,x46,x47,x48,x49,x50,x51,x52,x53,x54,x55,x56,x57,x58,x59,x60,x61,x62,x63,x64,x65,x66,x67,x68,x69,x70,x71,x72,x73,x74,x75,x76,x77,x78,x79,x80,x81,x82,x83
Company,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1,0,1.2810,0.022934,0.87454,1.21640,0.060940,0.188270,0.52510,0.018854,0.182790,0.006449,0.85822,2.00580,0.125460,6.9706,4.6512,0.050100,2.1984,0.018265,0.024978,0.027264,1.41730,9.5554,0.148720,0.66995,214.760,12.641,6.4607,0.043835,0.204590,0.35179,8.3161,0.28922,0.76606,2.5825,77.400,0.026722,1.63070,0.015016,0.005478,0.12730,9.6951,-0.73622,0.98559,0.180160,1.50060,0.026224,7.0513,1174.90,5.3399,0.85128,12.837,0.061737,0.180900,209.87,-0.582550,0.47101,0.109900,0.000000,0.000000,0.22009,7.1241,15.3810,3.2702,17.8720,34.6920,30.087,12.8,7991.4,364.9500,15.8,61.476,4.0,36.0,85.437,27.07,26.102,16.000,16.0,0.2,22,0.060390,30,49
1,0,1.2700,0.006454,0.82067,1.00490,-0.014080,0.181040,0.62288,0.006423,0.035991,0.001795,0.85152,-0.48644,0.179330,4.5764,3.7521,-0.014011,2.4575,0.027558,0.028804,0.041102,1.18010,7.2952,0.056026,0.67048,38.242,12.877,5.5506,0.265480,0.150190,0.41763,9.5276,0.41561,0.81699,2.6033,95.947,0.007580,0.83754,0.027425,0.045434,0.13774,5.6035,-0.64385,1.30190,0.046857,1.00950,0.007864,4.6022,1062.50,3.7389,0.94397,12.881,-0.000565,0.056298,250.14,-0.474770,0.38599,0.369330,0.000000,0.000000,0.00000,7.4166,7.1050,14.3210,18.7700,124.7600,26.124,11.8,8322.8,0.1896,15.6,24.579,0.0,36.0,107.090,31.31,30.194,17.000,16.0,0.4,22,0.010636,31,50
1,0,1.0529,-0.059379,0.92242,0.72926,0.020476,0.044865,0.43292,-0.081423,-0.765400,-0.054324,0.89314,0.41220,0.077578,11.8900,2.4884,0.028077,1.3957,0.012595,0.068116,0.014847,0.81652,7.1204,0.065220,0.84827,-498.390,13.225,16.2540,0.416570,0.074149,0.36723,9.3513,0.50356,0.91962,1.4931,144.670,-0.066483,0.95579,0.017270,0.028059,0.10242,9.4003,-14.03200,0.75746,-0.579760,0.57832,-0.064373,11.9880,651.15,10.9340,0.93478,12.909,0.041625,0.047562,280.55,-1.000000,0.48844,0.053299,0.003785,0.005191,0.00000,3.6373,7.0213,1.1538,9.8951,6.4467,30.245,10.3,8747.0,11.9460,15.2,20.700,0.0,35.0,120.870,36.07,35.273,17.000,15.0,-0.2,22,-0.455970,32,51
1,1,1.1131,-0.015229,0.85888,0.80974,0.076037,0.091033,0.67546,-0.018807,-0.107910,-0.065316,0.89581,0.99490,0.141120,6.0862,1.6382,0.093904,2.0588,0.011601,0.094385,0.014415,0.90391,7.9828,0.125160,0.80478,-75.867,13.305,8.8950,0.083774,0.054098,0.54360,7.0909,0.67133,0.93701,2.3533,219.750,-0.017000,0.38335,0.014327,0.203370,0.10143,5.7379,0.72205,1.39120,-0.150130,0.64508,-0.017731,6.1114,703.04,5.7028,0.87484,13.094,0.108400,0.101350,413.74,0.565000,0.34408,0.073356,0.000037,0.000045,0.00000,5.1442,9.9099,2.0408,-1.4903,-21.9070,34.285,11.5,9042.5,-18.7480,10.4,47.429,4.0,33.0,54.806,39.80,38.377,17.167,16.0,5.6,22,-0.325390,33,52
2,0,1.0623,0.107020,0.81460,0.83593,0.199960,0.047800,0.74200,0.128030,0.577250,0.094075,0.81549,3.01470,0.185400,4.3938,1.6169,0.239210,3.0311,0.006814,0.079346,0.008876,1.02510,4.7463,0.266020,0.76770,1423.100,11.575,17.4880,0.620770,0.046907,0.56963,9.4861,0.68143,0.94242,4.1296,222.650,0.131230,0.25301,0.008152,0.353010,0.17612,4.5088,-0.11339,1.05270,0.607660,0.25782,0.131380,4.4151,2465.40,4.1408,0.73398,11.396,0.250310,0.222370,315.34,-0.060101,0.20242,1.229100,-0.002491,-0.002980,0.22688,7.1241,15.3810,3.2702,17.8720,34.6920,30.087,12.8,7991.4,364.9500,15.8,61.476,4.0,36.0,85.437,27.07,26.102,16.000,16.0,0.2,29,1.251000,7,27


#### Exploration

In [44]:
financial_distress_data.shape

(3672, 84)

In [45]:
financial_distress_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3672 entries, 1 to 422
Data columns (total 84 columns):
Financial Distress    3672 non-null int64
x1                    3672 non-null float64
x2                    3672 non-null float64
x3                    3672 non-null float64
x4                    3672 non-null float64
x5                    3672 non-null float64
x6                    3672 non-null float64
x7                    3672 non-null float64
x8                    3672 non-null float64
x9                    3672 non-null float64
x10                   3672 non-null float64
x11                   3672 non-null float64
x12                   3672 non-null float64
x13                   3672 non-null float64
x14                   3672 non-null float64
x15                   3672 non-null float64
x16                   3672 non-null float64
x17                   3672 non-null float64
x18                   3672 non-null float64
x19                   3672 non-null float64
x20             

In [46]:
financial_distress_data.index

Int64Index([  1,   1,   1,   1,   2,   2,   2,   2,   2,   2,
            ...
            421, 421, 422, 422, 422, 422, 422, 422, 422, 422],
           dtype='int64', name='Company', length=3672)

In [47]:
financial_distress_data.columns

Index(['Financial Distress', 'x1', 'x2', 'x3', 'x4', 'x5', 'x6', 'x7', 'x8',
       'x9', 'x10', 'x11', 'x12', 'x13', 'x14', 'x15', 'x16', 'x17', 'x18',
       'x19', 'x20', 'x21', 'x22', 'x23', 'x24', 'x25', 'x26', 'x27', 'x28',
       'x29', 'x30', 'x31', 'x32', 'x33', 'x34', 'x35', 'x36', 'x37', 'x38',
       'x39', 'x40', 'x41', 'x42', 'x43', 'x44', 'x45', 'x46', 'x47', 'x48',
       'x49', 'x50', 'x51', 'x52', 'x53', 'x54', 'x55', 'x56', 'x57', 'x58',
       'x59', 'x60', 'x61', 'x62', 'x63', 'x64', 'x65', 'x66', 'x67', 'x68',
       'x69', 'x70', 'x71', 'x72', 'x73', 'x74', 'x75', 'x76', 'x77', 'x78',
       'x79', 'x80', 'x81', 'x82', 'x83'],
      dtype='object')

In [48]:
financial_distress_data.count()

Financial Distress    3672
x1                    3672
x2                    3672
x3                    3672
x4                    3672
x5                    3672
x6                    3672
x7                    3672
x8                    3672
x9                    3672
x10                   3672
x11                   3672
x12                   3672
x13                   3672
x14                   3672
x15                   3672
x16                   3672
x17                   3672
x18                   3672
x19                   3672
x20                   3672
x21                   3672
x22                   3672
x23                   3672
x24                   3672
x25                   3672
x26                   3672
x27                   3672
x28                   3672
x29                   3672
                      ... 
x54                   3672
x55                   3672
x56                   3672
x57                   3672
x58                   3672
x59                   3672
x

In [49]:
financial_distress_data.describe()

,Financial Distress,x1,x2,x3,x4,x5,x6,x7,x8,x9,x10,x11,x12,x13,x14,x15,x16,x17,x18,x19,x20,x21,x22,x23,x24,x25,x26,x27,x28,x29,x30,x31,x32,x33,x34,x35,x36,x37,x38,x39,x40,x41,x42,x43,x44,x45,x46,x47,x48,x49,x50,x51,x52,x53,x54,x55,x56,x57,x58,x59,x60,x61,x62,x63,x64,x65,x66,x67,x68,x69,x70,x71,x72,x73,x74,x75,x76,x77,x78,x79,x80,x81,x82,x83
count,3672.000000,3672.000000,3672.000000,3672.000000,3.672000e+03,3672.000000,3672.000000,3672.000000,3672.000000,3672.000000,3672.000000,3672.000000,3.672000e+03,3672.000000,3672.000000,3.672000e+03,3672.000000,3.672000e+03,3672.000000,3672.000000,3672.000000,3672.000000,3.672000e+03,3672.000000,3672.000000,3672.000000,3672.000000,3672.000000,3672.000000,3672.000000,3672.000000,3.672000e+03,3672.000000,3672.000000,3.672000e+03,3.672000e+03,3672.000000,3672.000000,3672.000000,3672.000000,3672.000000,3672.000000,3672.000000,3672.000000,3672.000000,3672.000000,3672.000000,3.672000e+03,3672.000000,3672.000000,3672.000000,3672.000000,3672.000000,3672.000000,3672.000000,3672.000000,3672.000000,3672.000000,3672.000000,3672.000000,3672.000000,3672.000000,3672.000000,3672.000000,3672.000000,3672.000000,3672.000000,3672.000000,3672.000000,3672.000000,3672.000000,3672.000000,3672.000000,3672.000000,3672.000000,3672.000000,3672.000000,3672.000000,3672.000000,3672.000000,3672.000000,3672.000000,3672.000000,3672.000000
mean,0.037037,1.387820,0.129706,0.615769,8.681599e-01,0.154949,0.106717,0.784031,39.274361,0.332610,0.136263,0.638835,1.252730e+05,0.384230,2.438322,8.185969e+03,-25.039036,2.058577e+03,0.041380,77.549137,0.103999,1.449663,1.419506e+01,0.273236,0.532118,996.000716,13.288179,-77.482669,0.202575,0.083651,0.354824,3.580871e+05,379.867463,0.863141,2.859282e+03,6.887839e+04,0.240324,0.312063,15.149440,120.237901,0.260097,3.033581,0.427294,11.526579,3.321408,0.149245,0.298761,1.490890e+03,2484.105343,2.126260,0.726764,12.929332,-21.252442,0.207304,3411.267857,0.007702,0.237413,1.027090,-0.010018,-6.869395,0.113802,2.694738,3.456374,1.299225,-0.703818,31.677402,28.245979,11.459051,15874.639434,21.472377,17.863013,23.301360,1.923501,30.407166,86.839822,91.920506,89.115908,17.780855,15.198708,-2.664305,19.714597,1.100488,13.122277,33.044935
std,0.188878,1.452926,0.120013,0.177904,5.719519e-01,0.124904,0.210555,1.033606,4305.688039,0.346135,0.138978,0.201986,1.468207e+06,0.177904,2.377307,1.086725e+05,1231.623609,6.022926e+04,0.045379,3256.469121,0.231024,0.968705,2.272613e+02,0.171174,0.182469,1822.605137,1.614000,4759.884902,0.285898,0.092991,0.170703,8.275936e+06,10444.296575,0.140751,7.633068e+04,2.337751e+06,0.294840,0.504356,420.164266,5128.434481,0.184866,3.187030,14.291208,136.307181,58.382438,0.764287,0.712088,4.558637e+04,2927.820444,2.162011,0.171174,1.761527,1209.468675,0.130165,14441.801918,0.416617,0.139899,25.636779,0.026567,290.276732,0.089494,3.922626,6.953459,8.738037,12.321989,43.852586,6.146999,0.952858,10026.105024,97.191938,7.119361,17.834175,1.469196,3.714512,16.706209,64.656504,64.349382,2.040152,2.828648,8.192663,7.508588,2.666733,9.465907,13.714563
min,0.000000,0.075170,-0.258080,0.016135,5.350000e-07,-0.269790,-0.627750,0.035160,-145000.000000,-3.611200,-0.318660,0.021491,-2.620000e+06,0.032101,0.016400,3.540000e-06,-35758.000000,0.000000e+00,0.000102,0.000000,0.000127,0.000003,7.730000e-07,-0.553850,0.016135,-3374.600000,8.195100,-288000.000000,-0.499440,0.000000,0.002082,0.000000e+00,0.010060,0.049372,4.560000e-05,0.000000e+00,-4.856200,0.000000,0.000244,-1.000000,0.000170,0.000002,-414.710000,0.089251,-56.719000,-9.376900,-1.074900,1.640100e-02,533.070000,0.016400,0.000000,0.000000,-33563.000000,-0.225220,0.010753,-1.000000,0.000000,-0.998710,-0.279260,-15649.000000,-0.349060,-7.714000,-11.824000,-25.736000,-21.411000,-21.907000,15.916000,10.300000,7941.800000,-58.122000,10.400000,-10.665000,0.000000,22.000000,54.806000,24.318000,23.776000,15.250000,12.000000,-20.200000,1.000000,-0.499220,1.000000,2.000000
25%,0.000000,0.952145,0.048701,0.501888,5.525575e-01,0.070001,

#### Train Test Split

In [50]:
feature_space = financial_distress_data.iloc[:, financial_distress_data.columns != 'Financial Distress']
feature_class = financial_distress_data.iloc[:, financial_distress_data.columns == 'Financial Distress']


training_set, test_set, class_set, test_class_set = train_test_split(feature_space,
                                                                    feature_class,
                                                                    test_size = 0.30, 
                                                                    random_state = 38)

#### Fit and Train the Random Forest Model

In [59]:
rf_model = RandomForestClassifier(random_state=38)

In [60]:
rf_model.fit(training_set,class_set)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=1,
            oob_score=False, random_state=38, verbose=0, warm_start=False)

#### Prediction and Evaulation

In [61]:
predictions = rf_model.predict(test_set)

In [62]:
from sklearn.metrics import classification_report,confusion_matrix

In [67]:
cm = confusion_matrix(test_class_set,predictions)

In [64]:
print(classification_report(test_class_set,predictions))

             precision    recall  f1-score   support

          0       0.97      0.99      0.98      1066
          1       0.53      0.22      0.31        36

avg / total       0.96      0.97      0.96      1102



In [68]:
predacc=(cm[0,0]+cm[1,1])/(cm[0,0]+cm[0,1]+cm[1,0]+cm[1,1])
predacc

0.9682395644283122

In [65]:
## End